In [104]:
from selectinf.lasso import lasso
from selectinf.Utils.base import selected_targets, selected_targets_WCLS

import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer

In [96]:
AW_df = pd.read_csv('~/Documents/git/SI-MRT/realdata/AW_df.csv')
# AW_df.head()
variables = [
    'ParticipantIdentifier',
    'Value_tran',
    'Notification_c',
    'TimeEnrolled_days',
    'Phases',
    'Value_30min_before',
    'NotificationType',
    'Baseline_steps',
    'IsWeekend',
    'IsIndoor',
    'IsLossFramed',
    'IsSnow',
    'IsActivity',
    'Value_30min_before_tran',
    'AgeEnrollment_years',
    'Gender',
    'Race',
    'ExerciseTimeAgg_min',
    'WalkDistanceAgg_m',
    'StepsAgg_priorweek',
    'Value_tran_sd_week',
    'Value_tran_sd_priorweek',
    'Distance_m_0'
]

#data = AW_df[variables].dropna()
data = AW_df[variables].replace([np.inf, -np.inf], np.nan).dropna()

## Adding binary columns for NotificationType, Gender, and Race based on conditions
# data['Naf'] = (data['NotificationType'] == 'afternoon').astype(int)
# data['Nev'] = (data['NotificationType'] == 'evening').astype(int)
# data['Nlu'] = (data['NotificationType'] == 'lunch').astype(int)
# data['Nmo'] = (data['NotificationType'] == 'morning').astype(int)

# data['IsMale'] = (data['Gender'] == 'Male').astype(int)
# data['IsCaucasian'] = (data['Race'] == 'Caucasian').astype(int)

# data['TPhase2'] = data['TimeEnrolled_days'] * (data['Phases'] == 2).astype(int)
# data['TPhase3'] = data['TimeEnrolled_days'] * (data['Phases'] == 3).astype(int)

# Remove original 'NotificationType' and 'Gender' columns
# data = data.drop(columns=['NotificationType', 'Gender', 'Race', 'Phases'])

# data = data.drop(columns=['TimeEnrolled_days'])


# Identify non-numeric columns in the dataset to ensure they are processed correctly
non_numeric_cols = data.select_dtypes(include=['object']).columns
# non_numeric_cols

# Apply one-hot encoding to the categorical columns and drop the original columns
data = pd.get_dummies(data, columns=['NotificationType', 'Gender', 'Race'], drop_first=True)
# data.convert_objects(convert_numeric=True)
data


# One-hot encode 'Phases'
# data['Phase1'] = (data['Phases'] == 1).astype(int)
# data['Phase2'] = (data['Phases'] == 2).astype(int)
# data['Phase3'] = (data['Phases'] == 3).astype(int)




# # Specified subset of variables for interaction with 'Phases'
# interaction_vars = [ 'Value_30min_before', 'Baseline_steps',
#     'IsWeekend', 'IsIndoor', 'IsSnow', 'IsActivity',
#     'Value_30min_before_tran', 'AgeEnrollment_years', 'IsMale', 'IsCaucasian',
#     'ExerciseTimeAgg_min', 'WalkDistanceAgg_m', 'StepsAgg_priorweek',
#     'Value_tran_sd_week', 'Value_tran_sd_priorweek', 'Distance_m_0', 'IsWeekend', 'Naf', 'Nev', 'Nmo']
# 
# # Adding interactions with each phase
# for var in interaction_vars:
#     for phase in ['Phase2', 'Phase3']:
#         interaction_col_name = f'{phase}_{var}'
#         data[interaction_col_name] = data[phase] * data[var]
        
# Remove original Phase columns
# data = data.drop(columns=['Phase1', 'Phase2', 'Phase3'])

,ParticipantIdentifier,Value_tran,Notification_c,TimeEnrolled_days,Phases,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,...,Value_tran_sd_week,Value_tran_sd_priorweek,Distance_m_0,NotificationType_evening,NotificationType_lunch,NotificationType_morning,Gender_M,Race_Caucasian,Race_Other,Race_Unknown
8,3,5.661223,-0.225066,50,2,223.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,False,True,False,True,False,False
9,3,5.596568,-0.225066,50,2,8.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,True,False,False,True,False,False
10,3,6.017376,-0.225066,50,2,438.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,False,False,False,True,False,False
11,3,5.671604,-0.225066,50,2,116.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,True,False,False,False,True,False,False
12,3,5.495117,0.774934,51,2,169.0,4037.571429,1,1,0,...,0.662054,0.760946,444.874503,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34066,925,5.618588,-0.225066,183,3,737.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False
34068,925,6.772508,-0.225066,183,3,533.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,True,False,False,False,False,False,False
34069,925,7.065186,-0.225066,184,3,77.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,False,True,False,False,False,False
34070,925,4.119037,-0.225066,184,3,194.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False


In [ ]:
import rpy2
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr
from sklearn import linear_model
# Activate automatic conversion between numpy arrays and R matrices/vectors
numpy2ri.activate()
selectiveInference = importr('selectiveInference')

import warnings
# suppress warnings
warnings.filterwarnings('ignore')

In [106]:
lambda_value = 0.5 * np.median(np.abs(X.T.dot(eps)).max(1))
# print(lambda_value)

# Fit the standard Lasso without an intercept
slf = linear_model.Lasso(alpha=lambda_value/n, fit_intercept=False)

slf.fit(X, Y)
# print(slf.coef_)
nonzerop = slf.coef_ != 0


# beta_targetp = np.linalg.pinv(X[:, nonzerop]).dot(X.dot(beta))
# print(beta_targetp)
# print(nonzerop.sum())        


# Apply Polyhedral method from selectiveInference package in R 
results = selectiveInference.fixedLassoInf(X, Y, slf.coef_, lam=lambda_value, sigma=np.sqrt(dispersion))
pvals = np.array(results.rx2('pval'))
pci = np.array(results.rx2('ci'))
# print(pci)


NameError: name 'selectiveInference' is not defined

In [116]:
from patsy import dmatrix

# Define knots for spline basis
knots = [data['TimeEnrolled_days'].quantile(q) for q in [0.25, 0.5, 0.75]]

# Create spline basis for TimeEnrolled_days
spline_basis = dmatrix(
    "bs(TimeEnrolled_days, knots=knots, degree=3, include_intercept=False)", 
    {"TimeEnrolled_days": data['TimeEnrolled_days']}, 
    return_type='dataframe'
)

# Rename spline columns
spline_basis.columns = [f"spline_TED_{i}" for i in range(spline_basis.shape[1])]

# Merge spline basis with data and drop the original TimeEnrolled_days
data = data.drop(columns=['TimeEnrolled_days']).join(spline_basis)

# Interact spline basis with other covariates to make spline-linear in Phase
for var in data.columns:
    if var not in ['ParticipantIdentifier', 'Value_tran', 'Notification_c']:
        for i in range(spline_basis.shape[1]):
            data[f"{var}_TED_{i}"] = data[var] * spline_basis[f"spline_TED_{i}"]

KeyError: 'TimeEnrolled_days'

In [21]:
data

,ParticipantIdentifier,Value_tran,Notification_c,TimeEnrolled_days,Phases,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,...,Value_tran_sd_week,Value_tran_sd_priorweek,Distance_m_0,NotificationType_evening,NotificationType_lunch,NotificationType_morning,Gender_M,Race_Caucasian,Race_Other,Race_Unknown
8,3,5.661223,-0.225066,50,2,223.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,False,True,False,True,False,False
9,3,5.596568,-0.225066,50,2,8.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,True,False,False,True,False,False
10,3,6.017376,-0.225066,50,2,438.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,False,False,False,False,True,False,False
11,3,5.671604,-0.225066,50,2,116.0,4037.571429,0,0,0,...,0.662054,0.760946,444.874503,True,False,False,False,True,False,False
12,3,5.495117,0.774934,51,2,169.0,4037.571429,1,1,0,...,0.662054,0.760946,444.874503,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34066,925,5.618588,-0.225066,183,3,737.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False
34068,925,6.772508,-0.225066,183,3,533.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,True,False,False,False,False,False,False
34069,925,7.065186,-0.225066,184,3,77.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,False,True,False,False,False,False
34070,925,4.119037,-0.225066,184,3,194.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False


In [97]:
np.random.seed(42019)

# Split data into estimation and rest set based on unique ParticipantIdentifier
unique_ids = data['ParticipantIdentifier'].unique()
est_ids = np.random.choice(unique_ids, size=int(len(unique_ids) * 1/3), replace=False)
est_set = data[data['ParticipantIdentifier'].isin(est_ids)]
rest_set = data[~data['ParticipantIdentifier'].isin(est_ids)]

# est_set = data.iloc[:6487]
# rest_set = data.iloc[6487:]

In [98]:
est_set
# data[covariates].select_dtypes(include='object').columns

,ParticipantIdentifier,Value_tran,Notification_c,TimeEnrolled_days,Phases,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,...,Value_tran_sd_week,Value_tran_sd_priorweek,Distance_m_0,NotificationType_evening,NotificationType_lunch,NotificationType_morning,Gender_M,Race_Caucasian,Race_Other,Race_Unknown
363,10,4.740575,0.774934,108,2,349.0,6143.285714,1,1,0,...,1.227172,1.535406,698.747947,False,True,False,True,True,False,False
364,10,7.033065,-0.225066,108,2,1365.0,6143.285714,0,0,0,...,1.227172,1.535406,698.747947,False,False,False,True,True,False,False
365,10,3.941582,-0.225066,108,2,9.0,6143.285714,0,0,0,...,1.227172,1.535406,698.747947,True,False,False,True,True,False,False
368,10,5.625821,-0.225066,112,2,436.0,6143.285714,0,0,0,...,1.227172,1.535406,698.747947,False,False,True,True,True,False,False
369,10,5.715382,-0.225066,112,2,817.0,6143.285714,0,0,0,...,1.227172,1.535406,698.747947,False,True,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34066,925,5.618588,-0.225066,183,3,737.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False
34068,925,6.772508,-0.225066,183,3,533.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,True,False,False,False,False,False,False
34069,925,7.065186,-0.225066,184,3,77.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,False,True,False,False,False,False
34070,925,4.119037,-0.225066,184,3,194.0,8030.714286,0,0,0,...,1.761142,1.404865,484.361600,False,True,False,False,False,False,False


In [99]:
# Columns to exclude
columns_to_remove = ['ParticipantIdentifier', 'Value_tran', 'Notification_c', 'AgeEnrollment_years']

# Define covariates for the model
covariates  = [col for col in data.columns if col not in columns_to_remove]

# Linear regression model for estimation set
X1 = est_set[covariates]
y1 = est_set['Value_tran']

alphahat = np.array(sm.OLS(y1, X1.astype(float)).fit().params)
# alphahat 

array([-4.59773388e-03,  6.11618311e-01, -2.97039208e-04,  3.01960267e-05,
       -1.40851449e-02, -3.23499497e-02, -1.88057071e-02,  4.86611411e-01,
        1.46305210e-01,  4.12591774e-01,  7.35398599e-04,  3.47340214e-05,
        6.16585341e-06,  1.98052719e-01,  5.59810634e-01,  1.13732016e-03,
        1.14251549e-01,  2.22489299e-01,  3.76080177e-01,  2.13472574e-01,
        4.36076013e-01,  0.00000000e+00,  5.32122436e-02])

In [100]:
X = rest_set[covariates]
Y = rest_set['Value_tran'] - np.dot(X,alphahat)
# X.loc[:, 'intercept'] = 1 

centered_treatment = rest_set['Notification_c']
X = X.multiply(centered_treatment, axis="index")

X=X.to_numpy(dtype='float64')
Y=Y.to_numpy(dtype='float64').reshape(-1)

In [101]:
print(Y.shape)
print(X.shape)
# Y.tolist()
# X['Y'] = y
# X['id'] = rest_set['ParticipantIdentifier']

(15542,)
(15542, 23)


In [160]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
# X
np.linalg.pinv(np.dot(X.T, X))
np.linalg.inv(np.dot(X.T, X))

array([[ 4.93526411e-10, -4.39207147e-12, -1.46529876e-09,
        -1.20299845e-09,  7.47899347e-09,  3.48947725e-08,
         1.68179786e-09, -1.14556943e-07,  4.02415953e-09,
        -5.20441008e-10, -1.53631282e-12, -2.49147509e-12,
         1.08230243e-08,  3.29447322e-08,  3.20303802e-10,
         1.42666989e-08,  1.67340385e-08, -1.90248465e-08,
        -2.92962882e-26,  1.22014103e-08,  2.84890193e-10,
         1.84723601e-10],
       [-4.39207147e-12,  1.21626459e-11,  1.44615775e-09,
        -3.95324922e-09, -2.33725532e-09, -5.62965156e-09,
         2.51113662e-09,  6.62532471e-10, -2.07418490e-10,
        -1.91614025e-10,  3.49033703e-13, -8.85741486e-12,
         2.80748357e-09,  2.76332213e-09, -2.67975350e-11,
         6.04792760e-10, -9.59752186e-10,  1.79483465e-09,
        -3.91744694e-25, -9.47914252e-09, -1.86050803e-11,
        -1.98743780e-11],
       [-1.46529876e-09,  1.44615774e-09,  1.72453114e-03,
        -9.94573084e-04, -2.49511026e-04, -1.06253271e-04,
    

In [ ]:
# Check for collinearity by calculating the variance inflation factor (VIF) for each covariate in X
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature in the X matrix
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# Display VIF values to examine multicollinearity among features
vif_data.sort_values(by="VIF", ascending=False)

In [40]:
df = pd.DataFrame({
    'id': rest_set['ParticipantIdentifier']
}).join(X)

df['Y'] = Y

AttributeError: 'numpy.ndarray' object has no attribute 'index'

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [102]:
n, p = X.shape
# N = len(df['id'].unique())
sigma_ = np.std(Y)

if n > (2 * p):
    dispersion = np.linalg.norm(Y - X.dot(np.linalg.pinv(X).dot(Y))) ** 2 / (n - p)
else:
    dispersion = sigma_ ** 2

eps = np.random.standard_normal((n, 2000)) * Y.std()
W = 50 * np.median(np.abs(X.T.dot(eps)).max(1))

const = lasso.gaussian
randomizer_scale = 1.
conv = const(X,
             Y,
             W,
             ridge_term=0.,
             randomizer_scale=randomizer_scale * np.sqrt(dispersion))

signs = conv.fit()
nonzero = signs != 0
print("size of selected set ", nonzero.sum())
print("potential moderators ", names[nonzero])

size of selected set  4
potential moderators                       x
2            IsWeekend
3             IsIndoor
11  StepsAgg_priorweek
12  Value_tran_sd_week


## Processed Data 

In [81]:
names=pd.read_csv('~/Documents/git/SI-MRT/realdata/names.csv')
names=names[:-1]
X=pd.read_csv('~/Documents/git/SI-MRT/realdata/Xoutput.csv')
Y=np.array(pd.read_csv('~/Documents/git/SI-MRT/realdata/Youtput.csv'))
Y=Y.ravel()
X=np.array(X.drop(columns=['outcome']))

In [71]:
alphahat=pd.read_csv('~/Documents/git/SI-MRT/realdata/alphahat.csv')
alphahat

,Estimate,Std. Error,t value,Pr(>|t|)
0,4.213725,0.189360,22.252491,2.083034e-106
1,0.000215,0.000044,4.862870,1.179608e-06
2,-0.143174,0.037029,-3.866576,1.112716e-04
3,0.041763,0.036331,1.149515,2.503790e-01
4,0.326207,0.039641,8.229052,2.193534e-16
5,0.000018,0.000009,2.094013,3.629101e-02
6,-0.004516,0.068823,-0.065616,9.476851e-01
7,-0.026323,0.058357,-0.451065,6.519550e-01
8,-0.192277,0.129017,-1.490326,1.361790e-01
9,-0.230617,0.343889,-0.670615,5.024858e-01


In [10]:
X=pd.read_csv('~/Documents/git/SI-MRT/realdata/Xoutput.csv')
X

,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,IsSnow,IsActivity,Value_30min_before_tran,AgeEnrollment_years,ExerciseTimeAgg_min,...,Distance_m_0,Naf,Nev,Nlu,Nmo,RaceC,GenderM,P2,P3,outcome
0,-13.053829,-1424.217790,0.0,0.0,0.0,0.0,0.0,-0.915800,-16.173289,-13.954093,...,-116.047013,-0.225066,0.000000,0.000000,0.000000,-0.225066,-0.225066,-19.355678,0.000000,-1.311566
1,-17.105018,-1424.217790,0.0,0.0,0.0,0.0,0.0,-0.976177,-16.173289,-13.954093,...,-116.047013,0.000000,-0.225066,0.000000,0.000000,-0.225066,-0.225066,-19.355678,0.000000,-0.915800
2,-42.537478,-1424.217790,0.0,0.0,0.0,0.0,0.0,-1.180334,-16.173289,-20.255942,...,-116.047013,0.000000,-0.225066,0.000000,0.000000,-0.225066,-0.225066,-19.580744,0.000000,-0.907970
3,-10.803169,-1424.217790,0.0,0.0,0.0,0.0,0.0,-0.873608,-16.173289,-0.900264,...,-116.047013,0.000000,0.000000,-0.225066,0.000000,-0.225066,-0.225066,-19.805810,0.000000,-1.174316
4,-31.284177,-1424.217790,0.0,0.0,0.0,0.0,0.0,-1.111391,-16.173289,-0.900264,...,-116.047013,-0.225066,0.000000,0.000000,0.000000,-0.225066,-0.225066,-19.805810,0.000000,-1.240431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15636,-165.873658,-1807.440921,0.0,0.0,0.0,0.0,0.0,-1.486171,-15.858692,-20.030876,...,-109.013339,0.000000,0.000000,-0.225066,0.000000,0.000000,0.000000,0.000000,-41.187082,-1.264553
15637,-119.960190,-1807.440921,0.0,0.0,0.0,0.0,0.0,-1.413293,-15.858692,-20.030876,...,-109.013339,0.000000,-0.225066,0.000000,0.000000,0.000000,0.000000,0.000000,-41.187082,-1.524261
15638,-17.330084,-1807.440921,0.0,0.0,0.0,0.0,0.0,-0.979100,-15.858692,-12.828763,...,-109.013339,0.000000,0.000000,0.000000,-0.225066,0.000000,0.000000,0.000000,-41.412148,-1.590133
15639,-43.662808,-1807.440921,0.0,0.0,0.0,0.0,0.0,-1.186195,-15.858692,-12.828763,...,-109.013339,0.000000,0.000000,-0.225066,0.000000,0.000000,0.000000,0.000000,-41.412148,-0.927055


In [8]:
from selectinf.lasso import lasso
from selectinf.Utils.base import selected_targets, selected_targets_WCLS

In [60]:
stepsdata = pd.read_csv('~/Documents/git/SI-MRT/realdata/stepsdata.csv')
columns = [stepsdata.columns[-1]] + list(stepsdata.columns[:-1])
stepsdata = stepsdata[columns]
stepsdata

,id,TimeEnrolled_days,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,IsSnow,IsActivity,Value_30min_before_tran,...,Value_tran_sd_week,Value_tran_sd_priorweek,Distance_m_0,Naf,Nev,Nlu,Nmo,P2,P3,Y
0,3,-11.253301,-50.189723,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.217475,...,-0.149006,-0.171263,-100.126135,-0.000000,-0.000000,-0.000000,-0.225066,-11.253301,-0.000000,-0.527985
1,3,-11.253301,-1.800528,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-0.481656,...,-0.149006,-0.171263,-100.126135,-0.000000,-0.000000,-0.225066,-0.000000,-11.253301,-0.000000,0.489688
2,3,-11.253301,-98.578918,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.369158,...,-0.149006,-0.171263,-100.126135,-0.225066,-0.000000,-0.000000,-0.000000,-11.253301,-0.000000,0.318160
3,3,-11.253301,-26.107659,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.070840,...,-0.149006,-0.171263,-100.126135,-0.000000,-0.225066,-0.000000,-0.000000,-11.253301,-0.000000,0.406279
4,3,39.521633,130.963842,3128.851287,0.774934,0.774934,0.0,0.0,0.774934,3.977622,...,0.513048,0.589683,344.748368,0.000000,0.000000,0.774934,0.000000,39.521633,0.000000,-0.103922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17607,925,-41.187082,-165.873658,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.486171,...,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.225066,-0.000000,-0.000000,-41.187082,-0.639202
17608,925,-41.187082,-119.960190,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.413293,...,-0.396373,-0.316187,-109.013339,-0.000000,-0.225066,-0.000000,-0.000000,-0.000000,-41.187082,0.888515
17609,925,-41.412148,-17.330084,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-0.979100,...,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.000000,-0.225066,-0.000000,-41.412148,1.037328
17610,925,-41.412148,-43.662808,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.186195,...,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.225066,-0.000000,-0.000000,-41.412148,-1.570377


In [61]:
covariates = [
    # 'TimeEnrolled_days',
    'Value_30min_before',
    'Baseline_steps',
    'IsWeekend',
    'IsIndoor',
    'IsLossFramed',
    'IsSnow',
    'IsActivity',
    'Value_30min_before_tran',
    'AgeEnrollment_years',
    'GenderM',
    'RaceC',
    'ExerciseTimeAgg_min',
    'WalkDistanceAgg_m',
    'StepsAgg_priorweek',
    'Value_tran_sd_week',
    'Value_tran_sd_priorweek',
    'Distance_m_0',
    'Naf',  
    'Nev',
    'Nlu',
    'Nmo',
    'P2',
    'P3'
]

# Linear regression model for estimation set
X = stepsdata[covariates]
Y = stepsdata['Y']

In [63]:
X

,Value_30min_before,Baseline_steps,IsWeekend,IsIndoor,IsLossFramed,IsSnow,IsActivity,Value_30min_before_tran,AgeEnrollment_years,GenderM,...,StepsAgg_priorweek,Value_tran_sd_week,Value_tran_sd_priorweek,Distance_m_0,Naf,Nev,Nlu,Nmo,P2,P3
0,-50.189723,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.217475,-12.024085,-0.0,...,-962.639530,-0.149006,-0.171263,-100.126135,-0.000000,-0.000000,-0.000000,-0.225066,-11.253301,-0.000000
1,-1.800528,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-0.481656,-12.024085,-0.0,...,-962.639530,-0.149006,-0.171263,-100.126135,-0.000000,-0.000000,-0.225066,-0.000000,-11.253301,-0.000000
2,-98.578918,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.369158,-12.024085,-0.0,...,-962.639530,-0.149006,-0.171263,-100.126135,-0.225066,-0.000000,-0.000000,-0.000000,-11.253301,-0.000000
3,-26.107659,-908.720141,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.070840,-12.024085,-0.0,...,-962.639530,-0.149006,-0.171263,-100.126135,-0.000000,-0.225066,-0.000000,-0.000000,-11.253301,-0.000000
4,130.963842,3128.851287,0.774934,0.774934,0.0,0.0,0.774934,3.977622,41.400615,0.0,...,3314.503327,0.513048,0.589683,344.748368,0.000000,0.000000,0.774934,0.000000,39.521633,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17607,-165.873658,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.486171,-15.858692,-0.0,...,-1679.185441,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.225066,-0.000000,-0.000000,-41.187082
17608,-119.960190,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.413293,-15.858692,-0.0,...,-1679.185441,-0.396373,-0.316187,-109.013339,-0.000000,-0.225066,-0.000000,-0.000000,-0.000000,-41.187082
17609,-17.330084,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-0.979100,-15.858692,-0.0,...,-1679.185441,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.000000,-0.225066,-0.000000,-41.412148
17610,-43.662808,-1807.440921,-0.000000,-0.000000,-0.0,-0.0,-0.000000,-1.186195,-15.858692,-0.0,...,-1679.185441,-0.396373,-0.316187,-109.013339,-0.000000,-0.000000,-0.225066,-0.000000,-0.000000,-41.412148


In [123]:
correlation_matrix = X.corr().abs()  # Get the absolute correlation matrix
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Identify columns with correlation greater than a threshold (e.g., 0.9)
collinear_pairs = [(column, index) for column, row in upper_triangle.iterrows() for index, value in row.items() if value > 0.95]

# Print collinear pairs
print("Collinear pairs (threshold > 0.99):", collinear_pairs)

Collinear pairs (threshold > 0.99): [('Baseline_steps', 'StepsAgg_priorweek'), ('Value_30min_before_tran', 'Distance_m_0')]


In [31]:
n, p = X.shape
sigma_ = np.std(Y)

if n > (2 * p):
    dispersion = np.linalg.norm(Y - X.dot(np.linalg.pinv(X).dot(Y))) ** 2 / (n - p)
else:
    dispersion = sigma_ ** 2

eps = np.random.standard_normal((n, 2000)) * Y.std()
W = 50 * np.median(np.abs(X.T.dot(eps)).max(1))

const = lasso.gaussian
randomizer_scale = 1.
conv = const(X,
             Y,
             W,
             ridge_term=0.,
             randomizer_scale=randomizer_scale * np.sqrt(dispersion))

signs = conv.fit()
nonzero = signs != 0
print("size of selected set ", nonzero.sum())
print("selected moderators ", names[nonzero])

size of selected set  6
selected moderators                       x
0   Value_30min_before
1       Baseline_steps
10   WalkDistanceAgg_m
11  StepsAgg_priorweek
14        Distance_m_0
22                  P3


In [55]:
# Generate random X (n by p) and Y (n by 1)
X = np.random.randn(n, p)
Y = np.random.randn(n, 1)

eps = np.random.standard_normal((n, 2000)) * Y.std()
W = 50 * np.median(np.abs(X.T.dot(eps)).max(1))

const = lasso.gaussian
randomizer_scale = 1.
conv = const(X,
             Y,
             W,
             ridge_term=0.,
             randomizer_scale=randomizer_scale * np.sqrt(dispersion))

signs = conv.fit()
nonzero = signs != 0

ValueError: operands could not be broadcast together with shapes (23,) (23,15641) (23,) 

In [10]:
conv.setup_inference(dispersion=dispersion)

# target_spec = selected_targets(conv.loglike,
#                                conv.observed_soln,
#                                dispersion=dispersion)

target_spec = selected_targets_WCLS(conv.loglike,
                                    data,
                                    conv.observed_soln,
                                    K = conv.K,
                                    dispersion= 1)

result_exact = conv.inference(target_spec)

intervals = np.asarray(result_exact[['lower_confidence', 'upper_confidence']])
lci = intervals[:, 0]
uci = intervals[:, 1]
print("check intervals ", lci, uci)


#means and sds of selected columns

print(np.mean(X[:,nonzero], axis=0))
print(np.std(X[:,nonzero], axis=0))

NameError: name 'data' is not defined